# Source: Rivals

In [24]:
#hide
import core_constants as cc
import functions as fx
import json

## Set Notebook Settings

In [25]:
conference = 'mwc'

years = cc.get_defYears()
headers= cc.get_header()
schoolsList = cc.get_schoolsList()
teamDirectory = cc.get_htmlDir('rivals', conference, 'teams')
playerDirectory = cc.get_htmlDir('rivals', conference, 'recruits')
#testDirectory = '..//tests//'

## Get & Save the Teams & Players Page HTML
#### Source: https://maryland.rivals.com/commitments/football/2012
> This page contains metadata of each player along with the Rivals ranking and stars.  Unlike 247Sports, we process the fetch and save of both pages directly from a single function

In [ ]:
fx.get_Rivals(conference, schoolsList, years, headers, sleepyTime=6)

centralmichigan: 2002
------------------------------
133235
129034
111906
112050
153218
154065
68029
129630
99696
144089
99704
162338
140590
36741
121584
kentstate: 2002
------------------------------
151081
toledo: 2002
------------------------------
92094
93302
30571
93051
westernmichigan: 2002
------------------------------
68598
32898
103150
64282
151232
93097
92704
129650
146489
centralmichigan: 2003
------------------------------
66952
92135
76273
91911
31788
92498
91895
91995
91917
47714
92687
91986
92016
56689
27306
91972
91973
91898
kentstate: 2003
------------------------------
91926
91905
77707
74674
75654
91999
91914
92688
91923
92004
84501
45491
91965
64740
92012
91930
91997
26899
91981
91979
92003
92691
61310
28818
91916
91900
31282
57324
91985
toledo: 2003
------------------------------
161864
63942
67833
58306
105008
85817
55880
92694
67790
92000
68093
127654
92239
71312
92534
51838
92013
134836
92006
91892
91988
westernmichigan: 2003
------------------------------
9197

50657
21721
85125
62586
60169
43145
19042
60017
23757
22363
84967
84962
46347
85123
85127
41201
44912
kentstate: 2013
------------------------------
98938
20727
67930
47489
98939
139389
130027
41021
48488
43610
64658
38526
43087
134746
75228
27674
19147
45683
65631
39866
20718
toledo: 2013
------------------------------
55929
26414
99067
42693
60529
14375
13987
83234
51873
99068
24928
26461
33069
13914
26449
12951
49849
18007
38669
100224
66771
26468
26452
westernmichigan: 2013
------------------------------
38215
20184
51464
79341
60125
104963
50117
61252
23081
17721
38527
82416
100644
33646
98972
61959
18651
83005
centralmichigan: 2014
------------------------------
54993
82411
28113
27885
51707
42180
82415
45256
82414
43316
53482
52759
26827
42196
63689
54175
82413
82412
20545
kentstate: 2014
------------------------------
49403
121302
81090
33454
70056
41602
71320
60009
53336
27671
16438
69906
79065
102559
74840
59408
22654
66816
toledo: 2014
------------------------------
82408
44

## Process Local Rivals HTML Files

> All of this processing is done locally, using the files saved in the previous few steps.  This creates an exhaustive store of all the fields grabbed from the scrapes.

In [26]:
cc.save_records('scrapedData', 'rivals_' + conference, fx.process_Rivals(playerDirectory, conference, schoolsList))

## Summarize the Dataset

> We don't need repetitive fields across the various datasets (ie - I don't need height coming back from 3 sources).  This step strips to only what I care about for the master print out.

In [27]:
outputDir = '..//summarizedData//'
dataset = 'rivals'

with open(outputDir + dataset + "data.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(fx.summarize_Rivals()))